In [1]:
%%info

## Map design comparison

In [2]:
death1 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_0.csv')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1575934167013_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def map_compare(death):
    # Create a header rdd for death data
    death_header = sc.parallelize(
        ['killed_by,killer_name,killer_placement,\
        killer_position_x,killer_position_y,map,match_id,\
        time,victim_name,victim_placement,victim_position_x,victim_position_y'])
    # Subtract the header row in death rdd
    death = death.subtract(death_header)
    # Calculate the number of games for each map
    death_map_count = death.map(lambda x: (x.split(',')[5], 1))
    death_map_group_count = death_map_count.reduceByKey(lambda x, y: x+y)
    # Get map:time pair
    death_map_live_count = death.map(
        lambda x: (x.split(',')[5], int(x.split(',')[7])))
    # Get the data for time less than 180
    death_map_live_count_180 = death_map_live_count.filter(
        lambda x: x[1] <= 180)
    death_map_live_count_180 = death_map_live_count_180.groupByKey()
    death_map_live_count_180 = death_map_live_count_180.mapValues(
        lambda x: len(x))
    death_map_live_final = death_map_live_count_180.join(death_map_group_count)
    # Get the ratio for time less than 180
    death_map_live_final_180 = death_map_live_final.map(
        lambda x: x[1]).map(lambda x: x[0]/x[1])
    # Get the data for time more than 180 and less than 600
    death_map_live_count_180_600 = death_map_live_count.filter(
        lambda x: x[1] <= 600).filter(lambda x: x[1] > 180)
    # Get the data for time more than 600 and less than 1200
    death_map_live_count_600_1200 = death_map_live_count.filter(
        lambda x: x[1] <= 1200).filter(lambda x: x[1] > 600)
    # Get the data for time more than 1200
    death_map_live_count_1200 = death_map_live_count.filter(
        lambda x: x[1] > 1200)
    # Get the count for time more than 180 and less than 600
    death_map_live_count_180_600 = death_map_live_count_180_600.groupByKey(
    ).mapValues(lambda x: len(x))
    # Get the count for time more than 600 and less than 1200
    death_map_live_count_600_1200 = death_map_live_count_600_1200.groupByKey(
    ).mapValues(lambda x: len(x))
    # Get the count for time more than 1200
    death_map_live_count_1200 = death_map_live_count_1200.groupByKey(
    ).mapValues(lambda x: len(x))
    death_map_live_final_180_600 = death_map_live_count_180_600.join(
        death_map_group_count)
    death_map_live_final_600_1200 = death_map_live_count_600_1200.join(
        death_map_group_count)
    death_map_live_final_1200 = death_map_live_count_600_1200.join(
        death_map_group_count)
    # Get the ratio for time less than 180_600
    death_map_live_final_180_600 = death_map_live_final_180_600.map(
        lambda x: x[1]).map(lambda x: x[0]/x[1])
    # Get the ratio for time less than 600_1200
    death_map_live_final_600_1200 = death_map_live_final_600_1200.map(
        lambda x: x[1]).map(lambda x: x[0]/x[1])
    # Get the ratio for time more than 1200
    death_map_live_final_1200 = death_map_live_final_1200.map(
        lambda x: x[1]).map(lambda x: x[0]/x[1])

    return death_map_live_final_180, death_map_live_final_180_600, death_map_live_final_600_1200, death_map_live_final_1200

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
death_map_live_final_180, death_map_live_final_180_600, death_map_live_final_600_1200, death_map_live_final_1200 = map_compare(
    death1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
death_map_live_final_180.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.16788648612109705, 0.18168057534517332, 0.1654980416872671]

In [7]:
death_map_live_final_180_600.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.3469940821857198, 0.3178359255473923, 0.32198110786862905]

In [8]:
death_map_live_final_600_1200.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.23679245862753412, 0.24171285865771974, 0.2444907153630453]

In [9]:
death_map_live_final_1200.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.23679245862753412, 0.24171285865771974, 0.2444907153630453]

In [ ]:
death_map_live_final_180.saveAsTextFile(
    's3n://chickenspark/deaths/death_map_live_final_180')

In [ ]:
death_map_live_final_180_600.saveAsTextFile(
    's3n://chickenspark/deaths/death_map_live_final_180_600')

In [ ]:
death_map_live_final_600_1200.saveAsTextFile(
    's3n://chickenspark/deaths/death_map_live_final_600_1200')

In [ ]:
death_map_live_final_1200.saveAsTextFile(
    's3n://chickenspark/deaths/death_map_live_final_1200')

## Aggregate data analysis

In [ ]:
agg1 = sc.textFile('s3a://chickenspark/aggregate/agg_match_stats_0.csv')

In [ ]:
print(agg1.count())  # count of obervations

In [ ]:
header = sc.parallelize(['date,game_size,match_id,match_mode,party_size,player_assists,\
                         player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,\
                         player_name,player_survive_time,team_id,team_placement'])
agg1 = agg1.subtract(header)

In [ ]:
# see which mode has longest average survive time
rdd1 = agg1.map(lambda x: (x.split(',')[3], x.split(',')[12]))

rdd1_1 = rdd1.filter(lambda x: isinstance(x[1], str))

rdd1_2 = rdd1_1.filter(lambda x: x[1].replace('.', '1').isnumeric())
rdd1_2 = rdd1_2.map(lambda x: (x[0], float(x[1])))

# remove the outliers
rdd1_2 = rdd1_2.filter(lambda x: x[1] < 10000)

### The average survive time in the TPP mode

In [ ]:
rdd1_3 = rdd1_2.groupByKey().map(lambda x: (
    x[0], sum(list(x[1])) / len(list(x[1]))))

In [ ]:
rdd2 = agg1.map(lambda x: x.split(',')[9]).filter(lambda x: isinstance(
    x, str)).filter(lambda x: x.replace('.', '1').isnumeric()).map(lambda x: float(x))

In [ ]:
rdd2.max()   # maximum damage (average 100-200 to kill a person)

In [ ]:
rdd2_1 = agg1.map(lambda x: x.split(',')[10]).filter(lambda x: isinstance(
    x, str)).filter(lambda x: x.replace('.', '1').isnumeric()).map(lambda x: float(x))
rdd2_1.max()   # maximum kill

In [ ]:
rdd2_2 = agg1.map(lambda x: x.split(',')[5]).filter(lambda x: isinstance(x, str)).filter(
    lambda x: x.replace('.', '1').isnumeric()).map(lambda x: float(x)).filter(lambda x: x < 10000)
rdd2_2.max()   # maximum assist

In [ ]:
rdd2_3 = agg1.map(lambda x: x.split(',')[12]).filter(lambda x: isinstance(x, str)).filter(
    lambda x: x.replace('.', '1').isnumeric()).map(lambda x: float(x)).filter(lambda x: x < 10000)
rdd2_3.max()  # maximum survive time (after deleting the outliers)

### Analyse the distribution of assistance, kills, survive times and damage by group-size.

In [ ]:
rdd3 = agg1.map(lambda x: (x.split(',')[4], (x.split(
    ',')[5], x.split(',')[9], x.split(',')[10], x.split(',')[12])))

In [ ]:
rdd3 = rdd3.filter(lambda x: x[1][3].replace('.', '1').isdigit() and x[1][0].replace(
    '.', '1').isdigit() and x[1][1].replace('.', '1').isdigit() and x[1][2].replace('.', '1').isdigit())

In [ ]:
rdd3_2 = rdd3.map(lambda x: (x[0], float(x[1][0]))).groupByKey()
# distribution of assistance

In [ ]:
# rdd3_2.saveAsTextFile('s3n://chickenspark/aggregate/temp/assistance_histogram')

In [ ]:
rdd3_3 = rdd3.map(lambda x: (x[0], float(x[1][2]))).groupByKey()

In [ ]:
# rdd3_3.saveAsTextFile('s3a://chickenspark/aggregate/temp/kill_histogram')

In [ ]:
rdd3_4 = rdd3.map(lambda x: (x[0], float(x[1][3]))).groupByKey()
# distribution of survive time

In [ ]:
# rdd3_4.saveAsTextFile('s3a://chickenspark/aggregate/temp/survive_histogram')

### For good-survivers, are they good at killing people or just hiding?

In [ ]:
# Because the avarage survive time is 786 seconds, we set the threshold as 1200 seconds (20 minutes) fro long-survivers.
rdd4 = agg1.map(lambda x: (x.split(',')[12], (x.split(',')[9], x.split(',')[10]))).filter(
    lambda x: x[0].replace('.', '1').isdigit() and float(x[0]) > 1200 and float(x[0]) < 10000)

In [ ]:
rdd4.count()  # sample has 3918402 of long-survivors.

In [ ]:
# the damage and killing number of long-survived-people distribution
rdd4_1 = rdd4.map(lambda x: float(x[1][0]))  # distribution of damage
rdd4_2 = rdd4.map(lambda x: float(x[1][1]))  # distributino of kills
rdd4_9 = rdd4.map(lambda x: (float(x[1][0]), float(x[1][1]))).map(
    lambda x: x[0]/(x[0]+1))

In [ ]:
# rdd4_1.saveAsTextFile('s3a://chickenspark/aggregate/temp/damage_histogram_for_long_surviver')
# rdd4_2.saveAsTextFile('s3a://chickenspark/aggregate/temp/kill_histogram_for_long_surviver')
# rdd4_9.saveAsTextFile('s3a://chickenspark/aggregate/temp/Kill_Damage_value_histogram_for_long_surviver')

### Compare with avarage damage and avarage kills off all

In [ ]:
# average damage and average killing overall:
rdd4_3 = agg1.map(lambda x: x.split(',')[9]).filter(
    lambda x: x.replace('.', '1').isdigit())
rdd4_4 = agg1.map(lambda x: x.split(',')[10]).filter(
    lambda x: x.replace('.', '1').isdigit())

In [ ]:
rdd4_3.map(lambda x: float(x)).mean()  # average damage overall is 123.9

In [ ]:
rdd4_4.map(lambda x: float(x)).mean()  # average kill number is  0.887

In [ ]:
# Damage/Kill value overrall: the smaller the value, more likely the person is.
rdd4_5 = agg1.map(lambda x: (x.split(',')[9], x.split(',')[10])).filter(lambda x: x[1].replace(
    '.', '1').isdigit() and x[0].replace('.', '1').isdigit()).map(lambda x: (float(x[0]), float(x[1])))
rdd4_5.map(lambda x: x[0]/(x[1]+1)).mean()
# average Damage/Kill value overall is 53.17

## Weapon Analysis

In [2]:
def whatweapon(death):
    header = sc.parallelize(
        ['killed_by,killer_name,killer_placement,\
        killer_position_x,killer_position_y,map,match_id,time,\
        victim_name,victim_placement,victim_position_x,victim_position_y'])
    noweapon = ['Down and Out', 'Falling', 'Hit by Car', 'Drown',
                'Uaz', 'RedZone', 'Motorbike (SideCar)', 'Motorbike',
                'Pickup Truck', 'Buggy', 'Boat', 'Van', 'death.RedZoneBomb_C', 'Aquarail', 'death.PG117_A_01_C']
    death = death.subtract(header)
    weapon = death.map(lambda x: (x.split(',')[0], 1))
    weapon_group = weapon.groupByKey()
    methods_kill = weapon_group.mapValues(
        lambda x: sum(x)).sortBy(lambda x: x[1], False)
    methods_kill = methods_kill.filter(lambda x: x[1] > 1)
    weapon_kill = methods_kill.filter(lambda x: x[0] not in noweapon)
    other_kill = methods_kill.filter(lambda x: x[0] in noweapon)
    return (weapon_kill, other_kill)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1575934167013_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
death1 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_0.csv')
weapon1, other1 = whatweapon(death1)
death2 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_1.csv')
weapon2, other2 = whatweapon(death2)
death3 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_2.csv')
weapon3, other3 = whatweapon(death3)
death4 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_3.csv')
weapon4, other4 = whatweapon(death4)
death5 = sc.textFile('s3a://chickenspark/deaths/kill_match_stats_final_4.csv')
weapon5, other5 = whatweapon(death5)
weapon_join1 = weapon1.join(weapon2)
weapon_join1 = weapon_join1.map(lambda x: (x[0], sum(x[1])))
weapon_join2 = weapon_join1.join(weapon3)
weapon_join2 = weapon_join2.map(lambda x: (x[0], sum(x[1])))
weapon_join3 = weapon_join2.join(weapon4)
weapon_join3 = weapon_join3.map(lambda x: (x[0], sum(x[1])))
weapon_join4 = weapon_join3.join(weapon5)
weapon_join4 = weapon_join4.map(lambda x: (
    x[0], sum(x[1]))).sortBy(lambda x: x[1], False)
other_join1 = other1.join(other2)
other_join1 = other_join1.map(lambda x: (x[0], sum(x[1])))
other_join2 = other_join1.join(other3)
other_join2 = other_join2.map(lambda x: (x[0], sum(x[1])))
other_join3 = other_join2.join(other4)
other_join3 = other_join3.map(lambda x: (x[0], sum(x[1])))
other_join4 = other_join3.join(other5)
other_join4 = other_join4.map(lambda x: (
    x[0], sum(x[1]))).sortBy(lambda x: x[1], False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…